In [ ]:
%pip install ncdump



In [ ]:
import datetime as dt
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import netCDF4
import netCDF4
import pandas as pd
import glob

In [ ]:
def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.

    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed

    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''
    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key

        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print ("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print( '\t\t%s:' % ncattr,\
                      repr(nc_fid.variables[key].getncattr(ncattr)))
        except KeyError:
            print( "\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print( "NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print( '\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print( "NetCDF dimension information:")
        for dim in nc_dims:
            print( "\tName:", dim )
            print( "\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print( "NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print( '\tName:', var)
                print( "\t\tdimensions:", nc_fid.variables[var].dimensions)
                print( "\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

In [ ]:
import netCDF4
import pandas as pd
import glob

def grab_files():
    '''
    Grabs files from folder
    '''

    file_list=glob.glob(r"cdf_files\**",recursive=True)
    file_list = file_list[1:]
    return file_list


def convert_files(file_list):
    '''
    Converts from .cdf files to .csv files 
    '''
    i = 1
    for x in file_list:

        print(f'Downloading {x}.....')
        nc = netCDF4.Dataset(x, mode='r')
        cols = list(nc.variables.keys())
        
        list_nc = []
        for c in cols:
            variable_data = nc.variables[c][:]
            if len(variable_data.shape) == 0:  # Check if the array is 0-dimensional
                list_nc.append([variable_data.item()])  # Convert scalar to list
            else:
                list_nc.append(list(variable_data))  # Convert other arrays to lists

        df_nc = pd.DataFrame(list_nc)
        df_nc = df_nc.T
        df_nc.columns = cols
        df_nc.to_csv(f"csv_files\{x[10:]}.csv", index=False)
        print(f'Downloaded! {i}/{len(file_list)}')
        i +=1
        return 
file_list = grab_files()
convert_files(file_list)